In [1]:
#import modules 
import numpy as np 
import pandas as pd
import statsmodels.api as sm 
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle
import numpy.ma as ma 
from pyproj import Transformer
import xarray as xr
import cmocean
import time 

### Load functions 
Two functions are necessary to run this notebook. The first function, "get_sia_seasonal_anomalies," calculates seasonal anomalies in the sea-ice area for an area of your choice. This area must lie within the Amundsen Sea. The second function, "sia_isv_regression," computes an Ordinary Least Squares (OLS) regression of the sea-ice-area anomalies and ice-shelf velocities. Ice-shelf velocity is the dependent variable while sea-ice-area anomaly is the independent variable. 

In [ ]:
# load sea-ice area seasonal anomaly function 
def get_sia_seasonal_anomalies(x1, y1, x2, y2, sic_arr, dates_arr, psx_arr, psy_arr):
    """
    Input PSXY coordinates, in km, of top left and bottom right corners of area of interest in the Amundsen Sea Embayment.\
    Output plot showing interest area in ASE, lists of summer and winter anomalies, and time labels for each season. 
    """
    start = time.time()
    
    ## find index of PSXY coordinates 
    xa = x1 * 1000
    diff_xa = np.absolute(psx_arr - xa)
    idx_x1 = diff_xa.argmin()
    
    ya = y1 * 1000
    diff_ya = np.absolute(psy_arr - ya)
    idx_y1 = diff_ya.argmin()
    
    xb = x2 * 1000
    diff_xb = np.absolute(psx_arr - xb)
    idx_x2 = diff_xb.argmin()
    
    yb = y2 * 1000
    diff_yb = np.absolute(psy_arr - yb)
    idx_y2 = diff_yb.argmin()
    
    xs = [idx_x1, idx_x2]
    ys = [idx_y1, idx_y2]
    
    area = sic_arr[min(ys):max(ys)+1, min(xs):max(xs)+1, :]
    row, col, depth = area.shape 
    
    # get psxy of area of interest for plotting 
    area_x = psx_arr[min(xs):max(xs)+1]
    area_y = psy_arr[min(ys):max(ys)+1]
    
    # get sea-ice area from sea-ice concentration 
    area_sia = np.empty((row, col, depth))
    area_sia[:] = np.nan 
    for i in range(depth):
        area_sia[:,:,i] = area[:,:,i]/100*12.5**2
        
    # get summer sea-ice area averages 
    summer_avgs = [] 
    summer_labels = []
    for j in range(min(dates_arr[:,0]), max(dates_arr[:,0])+1):
        if j == 2011:
            avg = np.nan
            summer_avgs = np.append(summer_avgs, avg)
            label = str(int(j)) + '-' + str(int(j+1))
            summer_labels = np.append(summer_labels, label)
        else: 
            idx = np.argwhere((dates_arr[:,1]>=10)&(dates_arr[:,0]==j)|(dates_arr[:,1]<=3)&(dates[:,0]==j+1))
            total = np.nansum(np.nansum(area_sia[:,:,idx]))
            avg = total / len(idx) 
            summer_avgs = np.append(summer_avgs, avg)
            label = str(int(j)) + '-' + str(int(j+1))
            summer_labels = np.append(summer_labels, label)
    summer_avgs = summer_avgs[:20] #cut off last avg bc data goes to 05/2022, no 2022-23 season 
    summer_labels = summer_labels[:20]
    
    # get summer anomalies 
    summer_series_avg = np.nanmean(summer_avgs)
    
    summer_anoms = []
    for i in range(len(summer_avgs)):
        anom = summer_avgs[i] - summer_series_avg
        summer_anoms = np.append(summer_anoms, anom)
    
    # get winter sea-ice area averages
    winter_monthly_avgs = [] 
    years = [] 
    
    for j in range(2003, max(dates_arr[:,0])+1):
        for i in range(4,9):
            idx = np.argwhere((dates_arr[:,1]==i)&(dates_arr[:,0]==j))
            total = np.nansum(np.nansum(area_sia[:,:,idx]))
            avg = total / len(idx)
            winter_monthly_avgs = np.append(winter_monthly_avgs, avg)
            years = np.append(years, int(j))
    
    winter_avgs = []
    winter_labels = [] 
    
    for j in range(2003, 2023):
        if j == 2011:
            avg = np.nan
            winter_avgs = np.append(winter_avgs, avg)
            winter_labels = np.append(winter_labels, str(j))
        elif j == 2012:
            avg = np.nan 
            winter_avgs = np.append(winter_avgs, avg)
            winter_labels = np.append(winter_labels, str(j))
        else: 
            idx = np.argwhere((years == j))
            total = np.nansum(winter_monthly_avgs[idx])
            avg = total/5
            winter_avgs = np.append(winter_avgs, avg)
            winter_labels = np.append(winter_labels, str(j))
    
    winter_avgs = winter_avgs[:19]
    winter_labels = winter_labels[:19]
    
    # get winter anomalies 
    winter_series_avg = np.nanmean(winter_avgs)
    
    winter_anoms = []
    for i in range(len(winter_avgs)):
        anom = winter_avgs[i] - winter_series_avg
        winter_anoms = np.append(winter_anoms, anom)
        
    ## plot interest area and its anomalies 
    area_plot = area[:,:, 10]
    ase_plot = sic_arr[:,:,10]
    
    # rectangle-area of interest 
    width = (psx_arr[idx_x2]/1000) - (psx_arr[idx_x1]/1000) 
    height = (psy_arr[idx_y1]/1000) - (psy_arr[idx_y2]/1000) 
    
    plt.figure(dpi = 200)
    plt.pcolormesh(psx_arr/1000, psy_arr/1000, ase_plot, cmap=cmocean.cm.ice, shading='nearest', vmin=0, vmax=100)
    cbar = plt.colorbar()
    cbar.set_label('Sea-ice concentration [%]')
    plt.pcolormesh(mask_int.x/1000, mask_int.y/1000, mask_int.data, cmap=cmocean.cm.gray, shading='nearest', vmin=1, vmax=4)
    plt.xlabel('PSX [km]')
    plt.ylabel('PSY [km]')
    plt.gca().add_patch(Rectangle((psx_arr[idx_x1]/1000,psy_arr[idx_y2]/1000), width, height, edgecolor='red', fill=False))
    plt.axis('equal')
    plt.show()
    
    # plot anomalies 
    summer_zeros = np.zeros(len(summer_avgs))
    winter_zeros = np.zeros(len(winter_avgs))
    
    plt.figure(dpi = 200)
    plt.plot(summer_labels, summer_anoms, 'crimson')
    plt.plot(summer_labels, summer_zeros, 'k--')
    plt.xticks(rotation=90)
    plt.ylim(top=np.nanmax(abs(summer_anoms))+1000, bottom=-np.nanmax(abs(summer_anoms))-1000)
    plt.ylabel('Sea-ice area [$km^2$]')
    plt.title('Summer sea-ice-area anomaly')
    plt.show()
    
    plt.figure(dpi=200)
    plt.plot(winter_labels, winter_anoms, 'midnightblue')
    plt.plot(winter_labels, winter_zeros, 'k--')
    plt.xticks(rotation=90)
    plt.ylim(top=np.nanmax(abs(winter_anoms))+1000, bottom=-np.nanmax(abs(winter_anoms))-1000)
    plt.ylabel('Sea-ice area [$km^2$]')
    plt.title('Winter sea-ice area anomaly')
    plt.show()
    
    return summer_anoms, summer_labels, winter_anoms, winter_labels

In [ ]:
def sia_isv_regression(anomaly_arr, velocity_list, season):
    '''
    This function reworks seasonal sia-anomaly and ice-shelf velocity data then inputs the data into an OLS regression model. 
    '''
    ## plot sia-anomaly against ice-shelf velocity 
    ice_shelf_v = velocity_list[1:] #exclude 2001-02 velocity bc no overlap w/sea-ice data 
    plt.figure(dpi=200)
    plt.plot(anomaly_arr, ice_shelf_v, 'black', marker='o', ls='none')
    plt.xlim(right=np.nanmax(abs(anomaly_arr))+1000, left=-np.nanmax(abs(anomaly_arr))-1000)
    plt.ylabel('Velocity [m/yr.]')
    plt.xlabel('SIA anomaly [$km^2$]')
    #plt.title('Ice-shelf velocity v. Sea-ice-area anomaly')
    plt.show()
    
    ## rework data & perform regression 
    anom_df = pd.DataFrame(anomaly_arr, columns = ['sia_anomaly']) 
    if season == 'summer': 
        sia_anoms = anom_df['sia_anomaly']
        del sia_anoms[9] #remove nan values 
        summer_sia_anoms = sia_anoms
        summer_sia_anoms = np.array(summer_sia_anoms).reshape((-1,1)) #reshape to input into model
        summer_sia_anoms = sm.add_constant(summer_sia_anoms) #add columns of ones for y-intercept calculation
        
        # remove velocity values to maintain overlap w/sea-ice data 
        del velocity_list[0]
        del velocity_list[10]
        summer_v_avgs = velocity_list 
        
        model = sm.OLS(summer_v_avgs, summer_sia_anoms) #create model 
        results = model.fit() #fit model 
        print(results.summary())
        
    elif season == 'winter':
        sia_anoms = anom_df['sia_anomaly']
        del sia_anoms[8]
        del sia_anoms[9]
        winter_sia_anoms = sia_anoms
        winter_sia_anoms = np.array(winter_sia_anoms).reshape((-1,1))
        winter_sia_anoms = sm.add_constant(winter_sia_anoms)
        
        del velocity_list[0:2]
        del velocity_list[10:12]
        winter_v_avgs = velocity_list
        
        model = sm.OLS(winter_v_avgs, winter_sia_anoms)
        results = model.fit()
        print(results.summary())
    else:
        print('ERROR: input season parameter')

In [ ]:
# Load sea ice data 
with np.load('ase_sic_12k.npz') as infile:
    data = np.ma.MaskedArray(data=infile['data'], mask=infile['mask'])
    psx = infile['x']
    psy = infile['y']
    dates = infile['dates']

# set values 110, 120 to NaN
row, col, depth = data.shape

sic = np.empty((row, col, depth))
sic[:] = np.nan

for i in range(0, depth):
        day = data[:,:,i]
        day[day == 110] = 'NaN' # masks missing data
        day[day == 120] = 'NaN' # masks land 
        day = day 
        sic[:,:,i] = day

# Load BedMachine Antarctica V2 and extract ice masks
bmpath = r'C:\Users\Keyleigh_Walliack\Documents\Amundsen_sea\Sea ice\BedMachineAntarctica_2020-07-15_v02.nc'

# Open the netCDF file
xr_ds = xr.open_dataset(bmpath)

mask = xr_ds.mask.squeeze()

# Convert ocean mask from 0 to nan
mask.data = mask.data.astype('float')
mask.data[mask.data == 0] = 'nan'
    
# Downsample and extract mask for ASE from BedMachine

# Find and round the extent of the ASE sic data to the nearest 1 km
xmin = np.floor(min(psx)/1000)*1000
xmax = np.round(max(psx)/1000)*1000
ymin = np.floor(min(psy)/1000)*1000
ymax = np.round(max(psy)/1000)*1000

# Interpolation interval, meters
dxy = 2500

# Resolution of the sic data, meters
dxy_sic = 12500

# x and y coordinates for interpolating the mask
xx = np.arange(xmin-dxy_sic/2, xmax+dxy_sic/2, dxy)
yy = np.arange(ymin-dxy_sic/2, ymax+dxy_sic/2, dxy)

# Interpolate the mask
mask_int = mask.interp(coords={'x': xx,'y': yy}, method = 'linear')

### Example: Pine Island Bay (PIB) sea ice v. Pine Island Glacier Ice Shelf (PIGIS) outer shelf velocity 

In [ ]:
# get seasonal sia-anomalies for PIB
summer_anoms,summer_dates,winter_anoms,winter_dates = get_sia_seasonal_anomalies(-1800, -325, -1500, -600, sic, dates, psx, psy)

In [ ]:
# load ice-shelf velocity data, edit path to csv file 
data = pd.read_csv(r'C:\Users\Keyleigh_Walliack\Desktop\Velocity_dataset\all_mean_values\TEIS_pp.csv')
v_pigis_ms = data['Velocity [m/yr.]']

In [ ]:
# find spearman's correlation coefficient
v = v_pigis_ms[1:]
res = stats.spearmanr(v, summer_anoms, nan_policy='omit')
print(res)

In [ ]:
# regression analysis
sia_isv_regression(summer_anoms, v_pigis_ms, 'summer')